In [1]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
!matplotlib inline
np.random.seed(0)

Using TensorFlow backend.
'matplotlib' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
episodes=pd.read_csv('episodes_cleaned.csv',index_col='Unnamed: 0')
scripts=pd.read_csv('scripts_cleaned.csv',index_col='Unnamed: 0')

episodes['Larry']=0
for ep in episodes.iterrows():
    names=(ep[1][4].replace(' and ',',').split(','))
    for name in names:
        if 'Larry David' in name:
            episodes['Larry'][ep[0]]=1
larry_eps=list(episodes.loc[episodes.Larry==1]['SEID'])
    

C:\kosta\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [5]:
" ".join(list(scripts.loc[scripts.SEID=='S01E01']['Dialogue']))

"so, im on line at the supermarket. two women in front of me. one of them, her total was eight dollars, the other three dollars. they both of course choose to pay by the use of the... cheque. cheque. now, the fact is, if its a woman in front of you thats writing the cheque, you will not be waiting long. i have noticed that women are very fast with cheques, yknow, cause they write out so many cheques. the keys, they can never find in their purse, they dont know where that is, but the cheque book they got that. they never fumble for the cheque book. the cheque book comes out of a holster (jerry draws imaginary book from a holster.) who do i make it out to? theres my id. theres something about a cheque that, to a man, is not masculine. i dont know exactly what it is. i think to a man, a cheque is like a note from your mother that says, i dont have any money, but if youll contact these people, im sure theyll stick up for me... if you just trust me this one time- i dont have any money but i

In [24]:
ep_corpus=dict()
for ep in episodes.SEID.values:
    ep_corpus[ep]=" ".join(list(scripts.loc[scripts.SEID==ep]['Dialogue']))
# for ep in episodes.SEID.values:
#     if ep in larry_eps:
#         ep_corpus[ep]['larry']

In [10]:
ep_corpus.keys()

dict_keys(['S01E00', 'S01E01', 'S01E02', 'S01E03', 'S01E04', 'S02E01', 'S02E02', 'S02E03', 'S02E04', 'S02E05', 'S02E06', 'S02E07', 'S02E08', 'S02E09', 'S02E10', 'S02E11', 'S02E12', 'S03E01', 'S03E02', 'S03E03', 'S03E04', 'S03E05', 'S03E06', 'S03E07', 'S03E08', 'S03E09', 'S03E10', 'S03E11', 'S03E12', 'S03E13', 'S03E14', 'S03E15', 'S03E16', 'S03E17', 'S03E18', 'S03E19', 'S03E20', 'S03E21', 'S03E22', 'S03E23', 'S04E01', 'S04E02', 'S04E03', 'S04E04', 'S04E05', 'S04E06', 'S04E07', 'S04E08', 'S04E09', 'S04E10', 'S04E11', 'S04E12', 'S04E13', 'S04E14', 'S04E15', 'S04E16', 'S04E17', 'S04E18', 'S04E19', 'S04E20', 'S04E21', 'S04E22', 'S04E23', 'S04E24', 'S05E01', 'S05E02', 'S05E03', 'S05E04', 'S05E05', 'S05E06', 'S05E07', 'S05E08', 'S05E09', 'S05E10', 'S05E11', 'S05E12', 'S05E13', 'S05E14', 'S05E15', 'S05E16', 'S05E17', 'S05E18', 'S05E20', 'S05E21', 'S05E22', 'S06E01', 'S06E02', 'S06E03', 'S06E04', 'S06E05', 'S06E06', 'S06E07', 'S06E08', 'S06E09', 'S06E10', 'S06E11', 'S06E12', 'S06E13', 'S06E16',

In [31]:
corpus_df=pd.DataFrame([ep_corpus.keys(),ep_corpus.values()])
corpus_df=corpus_df.transpose()
corpus_df.columns=['SEID','corpus']
corpus_df.head()

,SEID,corpus
0,S01E00,do you know what this is all about? do you kno...
1,S01E01,"so, im on line at the supermarket. two women i..."
2,S01E02,"so i move into the centre lane, now i get ahea..."
3,S01E03,"most men like working on things. tools, object..."
4,S01E04,went out to dinner the other night. check came...


In [32]:
corpus_df['larry']=episodes.Larry
corpus_df.head()

,SEID,corpus,larry
0,S01E00,do you know what this is all about? do you kno...,1
1,S01E01,"so, im on line at the supermarket. two women i...",1
2,S01E02,"so i move into the centre lane, now i get ahea...",0
3,S01E03,"most men like working on things. tools, object...",1
4,S01E04,went out to dinner the other night. check came...,1


In [43]:
len(corpus_df.corpus[0])

18534

In [44]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(corpus_df.corpus.values))
list_tokenized_train = tokenizer.texts_to_sequences(corpus_df.corpus.values)
X_t = pad_sequences(list_tokenized_train,maxlen=2000)
y=corpus_df.larry

In [45]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
X_resampled, y_resampled = RandomOverSampler().fit_sample(X_t, y) 

In [46]:
embedding_size = 128
input_ = Input(shape=(2000,))
x = Embedding(20000,embedding_size)(input_)
x = Bidirectional(LSTM(50,return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(100,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(50,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1,activation='sigmoid')(x)

model2 = Model(inputs=input_,outputs=x)
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

checkpoints_path = 'weights2_base.best.hdf5'
checkpoint = [ModelCheckpoint(checkpoints_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')]

history2=model2.fit(X_resampled,y_resampled,batch_size=32,epochs=6,validation_split=0.1, callbacks=checkpoint)

Train on 208 samples, validate on 24 samples
Epoch 1/6
208/208 [==============================] - 75s 363ms/step - loss: 0.6904 - acc: 0.5385 - val_loss: 0.7051 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.70507, saving model to weights2_base.best.hdf5
Epoch 2/6
208/208 [==============================] - 59s 282ms/step - loss: 0.6911 - acc: 0.5096 - val_loss: 0.7330 - val_acc: 0.0000e+00

Epoch 00002: val_loss did not improve from 0.70507
Epoch 3/6
208/208 [==============================] - 60s 287ms/step - loss: 0.6844 - acc: 0.5673 - val_loss: 0.7630 - val_acc: 0.0000e+00

Epoch 00003: val_loss did not improve from 0.70507
Epoch 4/6
208/208 [==============================] - 61s 294ms/step - loss: 0.6828 - acc: 0.5481 - val_loss: 0.7860 - val_acc: 0.0000e+00

Epoch 00004: val_loss did not improve from 0.70507
Epoch 5/6
208/208 [==============================] - 62s 300ms/step - loss: 0.6821 - acc: 0.5721 - val_loss: 0.8043 - val_acc: 0.0000e+00

Epoch 00005: va